In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder

# **Reading Input**

In [ ]:
path = '/kaggle/input/osic-pulmonary-fibrosis-progression/'

In [ ]:
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
sub = pd.read_csv(path + 'sample_submission.csv')

In [ ]:
train.head()

In [ ]:
print(train.shape)
print(test.shape)
print(sub.shape)

# Preparing Categorical Variables

In [ ]:
encoder = LabelEncoder()

In [ ]:
cat_features = ['Sex', 'SmokingStatus']
encoded = train[cat_features].apply(encoder.fit_transform)

In [ ]:
X = train[['Percent', 'Weeks', 'Age']].join(encoded)
X.head()

In [ ]:
Y = train['FVC']
Y.head()

# Linear Regression Model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=0)

In [ ]:
model = LinearRegression()
model.fit(X_train, Y_train)

In [ ]:
print(model.intercept_)
print(model.coef_)

In [ ]:
y_pred = model.predict(X_val)

# Making Predicitons

In [ ]:
sub.head()

In [ ]:
test.head()

In [ ]:
sub[['Patient', 'Weeks']] = sub.Patient_Week.str.split("_", expand=True)

In [ ]:
sub.head()

In [ ]:
encoded = test[cat_features].apply(encoder.fit_transform)
test.head()

In [ ]:
test2 = test[['Patient', 'Percent', 'Age']].join(encoded)
test2.head()

In [ ]:
sub = sub.drop('FVC', 1)
sub = sub.drop('Confidence', 1)
# test.drop('Weeks', 1)
sub = pd.merge(sub, test2, on='Patient', how='left')

In [ ]:
sub.head(100)

In [ ]:
X2 = sub[['Percent', 'Weeks', 'Age', 'Sex', 'SmokingStatus']]
sub['FVC'] = model.predict(X2)

In [ ]:
sub.head()

In [ ]:
sub['FVC_Group'] = sub.groupby(['Weeks','SmokingStatus','Sex','Age'])['FVC'].transform('mean')

In [ ]:
sub['Confidence'] = 100 * sub['FVC'] / sub['FVC_Group']
sub.head(100)

In [ ]:
submission = sub[['Patient_Week', 'FVC', 'Confidence']]
submission['FVC'] = submission['FVC'].astype(int)
submission['Confidence'] = submission['Confidence'].astype(int)
submission.head()

In [ ]:
submission.to_csv("/kaggle/working/submission.csv", index=False)